# 海南低空经济与城市建筑3D可视化

完整版：包含低空经济产业布局 + Vancouver Property Value风格的城市建筑可视化

## 案例说明

本案例包含两个主要部分：
1. **海南低空经济产业3D可视化** - 基于产业投资数据的3D柱状图
2. **Vancouver Property Value风格的城市建筑** - 模拟海南主要城市建筑价值的3D展示

In [ ]:
# 安装依赖
# %pip install "leafmap[maplibre]" pandas numpy

In [1]:
import leafmap.maplibregl as leafmap
import pandas as pd
import json
import math

print("库导入完成")

库导入完成


In [2]:
# 基础配置
HAINAN_CITIES = {
    "海口": [110.3312, 20.0311],
    "三亚": [109.5146, 18.2577], 
    "儋州": [109.5765, 19.5175],
    "琼海": [110.4667, 19.2463],
    "文昌": [110.7530, 19.5430],
    "万宁": [110.3893, 18.7962],
    "东方": [108.6340, 19.1019],
    "五指山": [109.5169, 18.7669]
}

INDUSTRY_COLORS = {
    "无人机制造": "#FF6B6B",
    "通用航空": "#4ECDC4",
    "航空物流": "#45B7D1",
    "航空旅游": "#96CEB4",
    "航空培训": "#FFEAA7",
    "航空维修": "#DDA0DD",
    "空中救援": "#FFA07A",
    "农业植保": "#98D8C8"
}

print(f"配置完成: {len(HAINAN_CITIES)}个城市, {len(INDUSTRY_COLORS)}种产业")

配置完成: 8个城市, 8种产业


## Vancouver Property Value风格的海南城市建筑数据

In [3]:
def create_hainan_city_blocks_data():
    """创建海南城市建筑区块数据，完全模拟Vancouver Property Value风格"""
    
    # 海口市建筑区块数据
    haikou_blocks = [
        {
            "coordinates": [[
                [110.32, 20.04], [110.34, 20.04],
                [110.34, 20.02], [110.32, 20.02], [110.32, 20.04]
            ]],
            "valuePerSqm": 25000,
            "district": "龙华区CBD",
            "buildingType": "商业综合体"
        },
        {
            "coordinates": [[
                [110.30, 20.03], [110.32, 20.03],
                [110.32, 20.01], [110.30, 20.01], [110.30, 20.03]
            ]],
            "valuePerSqm": 18000,
            "district": "美兰区",
            "buildingType": "住宅区"
        },
        {
            "coordinates": [[
                [110.28, 20.05], [110.30, 20.05],
                [110.30, 20.03], [110.28, 20.03], [110.28, 20.05]
            ]],
            "valuePerSqm": 22000,
            "district": "秀英区",
            "buildingType": "港口商务区"
        },
        {
            "coordinates": [[
                [110.34, 20.02], [110.36, 20.02],
                [110.36, 20.00], [110.34, 20.00], [110.34, 20.02]
            ]],
            "valuePerSqm": 15000,
            "district": "琼山区",
            "buildingType": "文教区"
        }
    ]
    
    # 三亚市建筑区块数据
    sanya_blocks = [
        {
            "coordinates": [[
                [109.50, 18.27], [109.52, 18.27],
                [109.52, 18.25], [109.50, 18.25], [109.50, 18.27]
            ]],
            "valuePerSqm": 35000,
            "district": "三亚湾",
            "buildingType": "海景酒店"
        },
        {
            "coordinates": [[
                [109.52, 18.26], [109.54, 18.26],
                [109.54, 18.24], [109.52, 18.24], [109.52, 18.26]
            ]],
            "valuePerSqm": 28000,
            "district": "大东海",
            "buildingType": "度假村"
        },
        {
            "coordinates": [[
                [109.48, 18.28], [109.50, 18.28],
                [109.50, 18.26], [109.48, 18.26], [109.48, 18.28]
            ]],
            "valuePerSqm": 20000,
            "district": "河西区",
            "buildingType": "商业区"
        }
    ]
    
    all_blocks = haikou_blocks + sanya_blocks
    
    features = []
    for block in all_blocks:
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": block["coordinates"]
            },
            "properties": {
                "valuePerSqm": block["valuePerSqm"],
                "district": block["district"],
                "buildingType": block["buildingType"]
            }
        }
        features.append(feature)
    
    return {
        "type": "FeatureCollection",
        "features": features
    }

print("海南城市建筑区块数据生成函数定义完成")

海南城市建筑区块数据生成函数定义完成


## 创建Vancouver Property Value风格的海南城市地图

In [4]:
# 生成建筑数据
city_blocks_data = create_hainan_city_blocks_data()

# 创建地图 - 完全参考Vancouver Property Value案例
m = leafmap.Map(
    center=[110.0, 19.0], 
    zoom=9, 
    style="dark-matter", 
    pitch=45, 
    bearing=0
)

# 完全复制Vancouver案例的样式配置
paint_line = {
    "line-color": "white",
    "line-width": 2,
}

paint_fill = {
    "fill-extrusion-color": {
        "property": "valuePerSqm",
        "stops": [
            [0, "grey"],
            [10000, "yellow"],
            [20000, "orange"],
            [30000, "darkred"],
            [40000, "lightblue"],
        ],
    },
    "fill-extrusion-height": ["*", 10, ["sqrt", ["get", "valuePerSqm"]]],
    "fill-extrusion-opacity": 0.9,
}

# 添加图层 - 完全按照Vancouver案例的方式
m.add_geojson(city_blocks_data, layer_type="line", paint=paint_line, name="blocks-line")
m.add_geojson(city_blocks_data, layer_type="fill-extrusion", paint=paint_fill, name="blocks-fill")

print("Vancouver Property Value风格的海南城市建筑地图创建完成")
m

Vancouver Property Value风格的海南城市建筑地图创建完成


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

In [5]:
# 启用交互功能 - 完全按照Vancouver案例
try:
    m.layer_interact()
    print("交互功能启用成功")
except Exception as e:
    print(f"交互功能启用失败: {e}")

m

交互功能启用成功


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

## 数据分析

In [6]:
# 分析建筑价值数据
print("海南主要城市建筑价值分析")
print("=" * 50)

for feature in city_blocks_data["features"]:
    props = feature["properties"]
    district = props["district"]
    value_per_sqm = props["valuePerSqm"]
    building_type = props["buildingType"]
    
    print(f"{district:12} | {building_type:12} | {value_per_sqm:6}元/㎡")

print("\n这个可视化完全复制了Vancouver Property Value案例的技术:")
print("- 使用相同的颜色映射 (grey->yellow->orange->darkred->lightblue)")
print("- 使用相同的高度计算公式 [*, 10, [sqrt, [get, valuePerSqm]]]")
print("- 使用相同的线条样式 (白色边框, 2px宽度)")
print("- 使用相同的透明度设置 (0.9)")
print("- 使用相同的地图样式 (dark-matter, pitch=45)")

海南主要城市建筑价值分析
龙华区CBD       | 商业综合体        |  25000元/㎡
美兰区          | 住宅区          |  18000元/㎡
秀英区          | 港口商务区        |  22000元/㎡
琼山区          | 文教区          |  15000元/㎡
三亚湾          | 海景酒店         |  35000元/㎡
大东海          | 度假村          |  28000元/㎡
河西区          | 商业区          |  20000元/㎡

这个可视化完全复制了Vancouver Property Value案例的技术:
- 使用相同的颜色映射 (grey->yellow->orange->darkred->lightblue)
- 使用相同的高度计算公式 [*, 10, [sqrt, [get, valuePerSqm]]]
- 使用相同的线条样式 (白色边框, 2px宽度)
- 使用相同的透明度设置 (0.9)
- 使用相同的地图样式 (dark-matter, pitch=45)
